# Homework 8
For this homework, _libsvm_ is needed. The easiest way to install this on _Mac OS X_ is using _Homebrew_ with the following command: `brew install libsvm`.

From this homework: questions 7, 8, 9 and 10 will be answered

In [1]:
from svmutil import *
import pandas as pd
from sklearn.cross_validation import KFold
from collections import Counter
import numpy as np

In [2]:
train = pd.read_table("features.train", sep=" +", header=None, engine='python')
train.columns = ["digit", "intensity", "symmetry"]
test = pd.read_table("features.test", sep=" +", header=None, engine='python')
test.columns = ["digit", "intensity", "symmetry"]

1-vs-5: Only keep data about the digits 1 and 5

In [17]:
filtered = train[(train.digit == 1) | (train.digit == 5)]
filtered["digit"].replace(1,float(-1),inplace=True)
filtered["digit"].replace(5,float(1),inplace=True)
filtered = filtered.reset_index(drop=True)

In [60]:
possible_C = [0.0001, 0.001, 0.01, 0.1, 1]
best_run_C = []
C_val_errors = {c: [] for c in possible_C}
for run in range(100):
    kf = KFold(len(filtered), n_folds=10, shuffle=True)
    C_errors = []
    for C in possible_C:
        fold_errors = []
        for train_index, val_index in kf:
            train_fold = filtered.ix[train_index]
            train_x = train_fold[["intensity", "symmetry"]].values.tolist()
            train_y = train_fold["digit"].values.tolist()
            val_fold = filtered.ix[val_index]
            val_x = val_fold[["intensity", "symmetry"]].values.tolist()
            val_y = val_fold["digit"].values.tolist()
            m = svm_train(train_y, train_x, '-q -t 1 -d 2 -c {} -r 1 -g 1'.format(C))
            p_label, p_acc, p_val = svm_predict(val_y, val_x, m, "-q")
            fold_errors.append((100-p_acc[0])/100.)
            if C == 0.001:
                print(p_acc)
        mean_fold_error = np.mean(fold_errors)
        C_val_errors[C].append(mean_fold_error)
        C_errors.append(mean_fold_error)
    best_run_C.append(possible_C[np.argmin(C_errors)])


(100.0, 0.0, 1.0)
(99.35897435897436, 0.10256410256410256, 0.9719525350593312)
(97.43589743589743, 0.41025641025641024, 0.8968295870746692)
(99.35897435897436, 0.10256410256410256, 0.9726315789473684)
(100.0, 0.0, 1.0)
(100.0, 0.0, 1.0)
(99.35897435897436, 0.10256410256410256, 0.9714285714285714)
(100.0, 0.0, 1.0)
(100.0, 0.0, 1.0)
(100.0, 0.0, 1.0)
(98.72611464968153, 0.20382165605095542, 0.9429920116194626)
(99.35897435897436, 0.10256410256410256, 0.9714285714285714)
(100.0, 0.0, 1.0)
(98.07692307692308, 0.3076923076923077, 0.9212121212121213)
(100.0, 0.0, 1.0)
(100.0, 0.0, 1.0)
(99.35897435897436, 0.10256410256410256, 0.9711431742508324)
(100.0, 0.0, 1.0)
(100.0, 0.0, 1.0)
(100.0, 0.0, 1.0)
(99.36305732484077, 0.10191082802547771, 0.9715166908563135)
(99.35897435897436, 0.10256410256410256, 0.9694477085781433)
(99.35897435897436, 0.10256410256410256, 0.9705215419501134)
(99.35897435897436, 0.10256410256410256, 0.9739130434782609)
(100.0, 0.0, 1.0)
(99.35897435897436, 0.1025641025641

## Question 7

In [61]:
best_c = Counter(best_run_C).most_common(1)[0][0]
print("Best C: ", best_c)

Best C:  0.001


## Question 8

In [62]:
print("Cross validation error when using C=", best_c, ": ", np.mean(C_val_errors[best_c]))

Cross validation error when using C= 0.001 :  0.00481659317328


In [26]:
filtered_test = test[(train.digit == 1) | (train.digit == 5)]
filtered_test["digit"].replace(1,float(-1),inplace=True)
filtered_test["digit"].replace(5,float(1),inplace=True)
filtered_test = filtered_test.reset_index(drop=True)

//anaconda/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [57]:
possible_C = [0.01, 1, 100, 10**4, 10**6]
E_ins = []
E_outs = []
for C in possible_C:
    train_x = filtered[["intensity", "symmetry"]].values.tolist()
    train_y = filtered["digit"].values.tolist()
    test_x = filtered_test[["intensity", "symmetry"]].values.tolist()
    test_y = filtered_test["digit"].values.tolist()
    m = svm_train(train_y, train_x, '-q -t 2 -d 2 -c {} -g 1'.format(C))
    train_label, train_acc, train_val = svm_predict(train_y, train_x, m, "-q")
    test_label, test_acc, test_val = svm_predict(test_y, test_x, m, "-q")
    E_ins.append((100-train_acc[0])/100.)
    E_outs.append((100-test_acc[0])/100.)

## Question 9

In [58]:
possible_C[np.argmin(E_ins)]

1000000

## Question 10

In [59]:
possible_C[np.argmin(E_outs)]

1000000